In [ ]:
import pandas as pd
import numpy as np
import urllib
import os
from zipfile import ZipFile
import json
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from matplotlib import cm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
%pylab inline

### 1. Data preparation

In [ ]:
!for ((y=93; y<=99; y+=1)); do wget http://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

!for ((y=0; y<=1; y+=1)); do wget http://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=0; y<=9; y+=1)); do wget http://www2.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=10; y<=15; y+=1)); do wget http://www2.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done

In [ ]:
#move zip files to PUIdata
os.system("mv zbp94totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp95totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp96totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp97totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp98totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp99totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp00totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp01totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp02totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp03totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp04totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp05totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp06totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp07totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp08totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp09totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp10totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp11totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp12totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp13totals.zip " + os.getenv("PUIDATA"))
os.system("mv zbp14totals.zip " + os.getenv("PUIDATA"))

### Business data

In [ ]:
#new data frame ZIP, EST, YEAR
years = range(1995,2015)
df = pd.DataFrame()
for year in years:
    abbr_year = str(year)[2:4]
    fname = 'zbp' + abbr_year + 'totals.zip'
    fullname = os.getenv("PUIDATA") + '/' + fname
    zf = ZipFile(fullname)
    # open and read file
    df_that_year = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    # uppercase, to make the columns in a uniform case
    df_that_year.columns = [item.upper() for item in df_that_year.columns] 
    df_that_year['YEAR'] = year
    # subset necessary columns
    df_sliced = df_that_year.loc[:,['ZIP','EST','YEAR']]
    # concat (merge) this year's dataframe to main dataframe
    df = pd.concat([df, df_sliced], axis=0)

In [ ]:
df.head()

### NYC zip codes shape file 

In [ ]:
with open(os.getenv("PUIDATA") + '/nyc-zip-code-tabulation-areas-polygons.geojson') as f:
    nyc_geojson = pd.DataFrame(json.load(f))

In [ ]:
zipcodes = []
polygons = []
for i in range(len(nyc_geojson)):
    zipcode = nyc_geojson.ix[i]['features']['properties']['postalCode']
    poly = nyc_geojson.ix[i]['features']['geometry']
    zipcodes.append(int(zipcode))
    polygons.append(poly)
nyc_shape = pd.DataFrame({'geometry': polygons, 'ZIP':zipcodes})

In [ ]:
nyc_shape.head()

### Merging 2 datasets to prepare the data for clustering. 

In [ ]:
df.index = df.ZIP

In [ ]:
df_NY = df.ix[zipcodes].dropna(how='any')

# convert ZIP and YEAR as int
df_NY['ZIP'] = df_NY['ZIP'].astype(int)
df_NY['YEAR'] = df_NY['YEAR'].astype(int)

In [ ]:
df_NY_pivot = df_NY.pivot_table(index='ZIP', columns='YEAR', values='EST')
df_NY_pivot.head()

In [ ]:
df_NY_pivot.shape

In [ ]:
df_NY_standardized = df_NY_pivot.dropna(how='any').copy()
for row in df_NY_standardized.index:
    row_values = df_NY_pivot.ix[row]
    row_mean = row_values.mean()
    row_std = row_values.std()
    df_NY_standardized.ix[row] = (row_values - row_mean)/row_std

### Standardize values 

In [ ]:
df_NY_standardized.head()

### 2. Cluster the NUMBER OF ESTABLISHMENTS time series with K-means 

In [ ]:
def elbow(data, K):
    '''calculation of average within-cluster
    sum of squares, use elbow method to determine number of clusters'''
    
    KM = [KMeans(n_clusters=k).fit(data) for k in K]
    centroids = [k.cluster_centers_ for k in KM]
    
    D_k = [cdist(data, cent, 'euclidean') for cent in centroids]
    globals()['dist'] = [np.min(D, axis=1) for D in D_k]
    avgWithinSS = [sum(d)/data.shape[0] for d in dist]
    
    # elbow curve
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avgWithinSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')
    plt.show()

In [ ]:
elbow(df_NY_standardized, range(1,30))

#### Plot 1: The elbow method for KMeans clustering

The elbow method shows 2 clusters. Choosed 5 as number of cluster as per instructions.    

In [ ]:
# implement k-means
km = KMeans(5).fit(df_NY_standardized)

In [ ]:
# store the result in 'kmeans' column
df_NY_standardized['kmeans'] = km.labels_

### 3. Pot the cluster centers 

In [ ]:
fig = plt.figure(figsize=(12,15))
ax1 = fig.add_subplot(3,2,1)
c = ['r','g','b','k','m']

# first subplot: all cluster centers
for i in range(5):
    ax1.scatter(years,km.cluster_centers_[i], c=c[i], 
                lw=0, alpha=0.5, s=70, label='cluster ' + str(i))
ax1.legend(loc='lower right', fontsize=8)
ax1.set_title('Cluster Centers', fontsize=15)
ax1.set_xlabel('year')
ax1.set_ylabel('cluster center values')
ax1.set_ylim(-3,4)

# other subplots: plot the timeseries and its cluster centers
for subplot_number in [2, 3, 4, 5, 6]:
    ax = fig.add_subplot(3, 2, subplot_number, sharex=ax1, sharey=ax1)
    features = df_NY_standardized.iloc[:, 0:20]
    cluster = subplot_number - 2
    features[df_NY_standardized['kmeans'] == cluster].T\
        .plot(legend=False, alpha=0.5, cmap='Greys', ax=ax, zorder=1)
    ax.scatter(years, km.cluster_centers_[cluster], c=c[cluster], zorder=2,
               lw=0, s=70)
    ax.set_title('cluster '+ str(cluster))
    ax.grid()
plt.tight_layout()

#### Plot 2: All cluster centers with their respective time series

All clusters have different trends.

### 4. Use another clustering algorithm 

In [ ]:
# select all features' data, store in a variable X
X = df_NY_standardized.iloc[:, 0:20]

In [ ]:
# linkage all observations
links = linkage(X, 'ward')

In [ ]:
# plot the dendrogram
plt.figure(figsize=(8, 32))
plt.title('Dendrogram', fontsize=20)
plt.xlabel('distance', fontsize=15)
dendrogram(links, leaf_font_size=10, orientation='left', labels=X.index)
plt.show()

#### Plot 3: Hierarchical Clustering Dendrogram

There are 2 main clusters based on the linkage (green and red)

In [ ]:
# pick only 5 clusters from hierarchical clustering
# store the labels into 'hierarchical' column in 'df_NY_standardized'

k = 5
df_NY_standardized['hierarchical'] = fcluster(links, k, criterion='maxclust')

### 5. Overlay data on a NYC map

Merging nyc_shape with k-means and hierarchical labels 

In [ ]:
# kmeans
df_labels = pd.DataFrame(df_NY_standardized['kmeans']).reset_index()
nyc_shape = pd.merge(nyc_shape,df_labels, how='left', on='ZIP')
nyc_shape.fillna(-1, inplace=True) # because first cluster start from 0

In [ ]:
# hierarchical
df_labels = pd.DataFrame(df_NY_standardized['hierarchical']).reset_index()
nyc_shape = pd.merge(nyc_shape,df_labels, how='left', on='ZIP')
nyc_shape.fillna(0, inplace=True) # because first cluster start from 1

In [ ]:
nyc_shape.tail()

In [ ]:
def plotNY(column_cluster, n_cluster, desired_axis):
    ''' This function is to plot polygons of NYC:
    column_cluster: choose between kmeans or hierarchical,
    n_cluster: number of cluster,
    desired_axis: selected ax'''
    
    ax = desired_axis
    colors = np.linspace(0, 1, n_cluster+1)
    cmap = cm.get_cmap('Blues')
    for i in range(len(nyc_shape)):
        poly = nyc_shape['geometry'][i]
        if (column_cluster == 'hierarchical') | (column_cluster == 'hierarchical2'):
            color_index = int(nyc_shape[column_cluster][i])
        else:
            color_index = int(nyc_shape[column_cluster][i]+1)
        color = cmap(colors[color_index])
        ax.add_patch(PolygonPatch(poly, fc=color, ec='k', zorder=2))
    ax.axis('scaled')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title(column_cluster + ' Clustering', fontsize=20)

In [ ]:
# plot kmeans cluster on NYC Map
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
plotNY('kmeans', 5, ax)

#### Plot 4. Kmeans on the NYC map 

In [ ]:
# plot hierarchical cluster on NYC Map
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
plotNY('hierarchical', 5, ax)

#### Plot 5. Hierarchical cluster on NYC Map

### 6. Compare the results of the 2 algorithms

In [ ]:
#plot 2 maps kmeans and hierarchical
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(221)
plotNY('kmeans', 5, ax1)
ax2 = fig.add_subplot(222)
plotNY('hierarchical', 5, ax2)
plt.tight_layout(w_pad=0.2, h_pad=0.2, pad=5)
plt.suptitle('Number of Cluster: 5', fontsize=20)

In [ ]:
km2 = KMeans(2).fit(df_NY_standardized.iloc[:, 0:20])
df_NY_standardized['kmeans2'] = km2.labels_
df_NY_standardized['hierarchical2'] = fcluster(links, 2, criterion='maxclust')

In [ ]:
# kmeans
df_labels = pd.DataFrame(df_NY_standardized['kmeans2']).reset_index()
nyc_shape = pd.merge(nyc_shape,df_labels, how='left', on='ZIP')
nyc_shape.fillna(-1, inplace=True) # because first cluster start from 0

# hierarchical
df_labels = pd.DataFrame(df_NY_standardized['hierarchical2']).reset_index()
nyc_shape = pd.merge(nyc_shape,df_labels, how='left', on='ZIP')
nyc_shape.fillna(0, inplace=True) # because first cluster start from 1

In [ ]:
nyc_shape.tail()

In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(221)
plotNY('kmeans2', 2, ax1)
ax2 = fig.add_subplot(222)
plotNY('hierarchical2', 2, ax2)
plt.tight_layout(w_pad=0.2, h_pad=0.2, pad=5)
plt.suptitle('Number of Cluster: 2', fontsize=20)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,3,1)
c = ['r','g']

# first subplot: all cluster centers
for i in range(2):
    ax1.scatter(years,km2.cluster_centers_[i], c=c[i], 
                lw=0, alpha=0.5, s=70, label='cluster ' + str(i))
ax1.legend(loc='lower right', fontsize=8)
ax1.set_title('Cluster Centers', fontsize=15)
ax1.set_xlabel('year')
ax1.set_ylabel('cluster center values')
ax1.set_ylim(-3,4)

for subplot_number in [2, 3]:
    ax = fig.add_subplot(1, 3, subplot_number, sharex=ax1, sharey=ax1)
    features = df_NY_standardized.iloc[:, 0:20]
    cluster = subplot_number - 2
    features[df_NY_standardized['kmeans2'] == cluster].T\
        .plot(legend=False, alpha=0.5, cmap='Greys', ax=ax, zorder=1)
    ax.scatter(years, km2.cluster_centers_[cluster], c=c[cluster], zorder=2,
               lw=0, s=70)
    ax.set_title('cluster '+ str(cluster))
    ax.grid()
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,5))

for subplot_number in [1, 2]:
    ax = fig.add_subplot(1, 3, subplot_number, sharex=ax1, sharey=ax1)
    features = df_NY_standardized.iloc[:, 0:20]
    cluster = subplot_number
    features[df_NY_standardized['hierarchical2'] == cluster].T\
        .plot(legend=False, alpha=0.5, cmap='Greys', ax=ax, zorder=1)
    ax.set_title('cluster '+ str(cluster))
    ax.grid()
plt.tight_layout()

In [ ]:
diff = df_NY_standardized[(df_NY_standardized['hierarchical2'] 
                           - df_NY_standardized['kmeans2']) != 1]
diff.iloc[:, 0:20].T.plot(legend=False, alpha=0.5, c='k',
                         title='All timeseries categorized differently by KMeans and Hierarchical',
                         figsize=(8,6))

In [ ]:
#@Review: Plots not rendered. why???
# Missing comparision and interprettion of the clustering algorithms.
